In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Carrega dados
df = pd.read_csv('/Users/franciscoabraao/Documents/Jubart_2024/Painel_Precos_diarios_morumbi/precos_carrefour_morumbi_last_30_days.csv')
df['data'] = pd.to_datetime(df['data'])

# Categorias principais
categorias = ["frango", "picanha", "tilápia", "camarão", "salmão", "bacalhau", "merluza", "ovos", "leite", "azeite"]

# Cria pasta de imagens
os.makedirs("imgs_categoria", exist_ok=True)

for categoria in categorias:
    subset = df[df['produto'].str.contains(categoria, case=False, na=False)]
    top_produtos = subset['produto'].value_counts().nlargest(10).index
    subset = subset[subset['produto'].isin(top_produtos)]

    if subset.empty:
        continue

    # Criação da figura com subplots
    fig, axs = plt.subplots(nrows=len(top_produtos), figsize=(4, 1.5 * len(top_produtos)))

    if len(top_produtos) == 1:
        axs = [axs]

    for ax, produto in zip(axs, top_produtos):
        historico = subset[subset['produto'] == produto].sort_values('data')
        datas = historico['data'].dt.strftime('%d').tolist()
        precos = historico['preco'].tolist()
        if not precos:
            continue

        ax.plot(precos, marker='o', linewidth=1.5, markersize=4)
        ax.set_xticks(range(len(datas)))
        ax.set_xticklabels(datas, fontsize=6)
        ax.set_xlabel("Dia", fontsize=7)
        ax.set_ylabel("R$", fontsize=7)
        ax.set_title(produto, fontsize=8)
        ax.tick_params(axis='y', labelsize=6)
        ax.grid(True, linestyle='--', linewidth=0.3)

    plt.tight_layout()
    plt.savefig(f"imgs_categoria/{categoria}.png", dpi=100)
    plt.close(fig)